In [1]:
from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, get_column_names
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window
from data_generator.constants import ColConstants

from pyspark.sql.functions import col, lower, lit, rand, array, floor, concat, when, row_number, concat_ws, date_format, count, split, size
  

/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/02/12 20:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [3]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [4]:
df = read_postgres_table("dob_age_raw_data")

In [5]:
# df.show()
get_row_count(df, True)

10000


10000

In [6]:
# df.where(col("Age") < 16).show()

In [7]:
from constants.admission_types_tests_dataset import admission_mapping, admission_tests
# Flatten the mapping and create a DataFrame
flattened = [
    (top_level, sub_level, stay_type, admission_tests.get(sub_level, ["No tests"]))
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level, stay_types in sub_level_dict.items()
    for stay_type in stay_types
]



In [8]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""
mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_type", "possible_tests"])

display_df(mapping_df, 100)

,top_level_admission,sub_level_admission,stay_type,possible_tests
0,emergency,injury_rtc,day_patient,"[X-rays, CT scans, MRI, Ultrasound, Blood tests]"
1,emergency,injury_rtc,inpatient,"[X-rays, CT scans, MRI, Ultrasound, Blood tests]"
2,emergency,self_inflicted,inpatient,"[Psychological assessment, X-rays (for physica..."
3,emergency,cardiology,day_patient,"[ECG, Echocardiogram, Stress tests, Cardiac ca..."
4,emergency,cardiology,inpatient,"[ECG, Echocardiogram, Stress tests, Cardiac ca..."
5,emergency,neurology,inpatient,"[MRI or CT scans of the brain, Electroencephal..."
6,emergency,pulmonology,day_patient,"[Pulmonary function tests, Chest X-ray, CT sca..."
7,emergency,pulmonology,inpatient,"[Pulmonary function tests, Chest X-ray, CT sca..."
8,emergency,infectious_diseases,outpatient,"[Blood cultures, PCR tests, Antibody tests, ch..."
9,emergency,infectious_diseases,inpatient,"[Blood cultures, PCR tests, Antibody tests, ch..."


In [9]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_types = list(admission_mapping.keys())

print(admission_types)

keys_array = array([lit(key) for key in admission_types])

# Define constants and conditions
female_only = ['maternity', 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65) & (col("sub_level_admission") == "geriatrics")

df = (df.withColumn("top_level_admission", keys_array[floor(rand() * len(admission_types))]).drop("admission_type")
        .withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
      )
get_row_count(df, True)
df = df.join(mapping_df, "top_level_admission", "inner")
get_row_count(df, True)
display_df(df, 999)

['emergency', 'gp_referral', 'hospital_referral', 'self_referral', 'elective']
10000
89939


,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,...,room_number,discharge_date,medication,test_results,DOB,is_female,is_pediatric,sub_level_admission,stay_type,possible_tests
0,elective,Antonio Frederick,85,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,...,480,2020-05-03,Penicillin,Abnormal,1938-09-16,False,False,maternity,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te..."
1,elective,Antonio Frederick,85,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,...,480,2020-05-03,Penicillin,Abnormal,1938-09-16,False,False,maternity,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
2,elective,Antonio Frederick,85,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,...,480,2020-05-03,Penicillin,Abnormal,1938-09-16,False,False,orthopedics,day_patient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
3,elective,Antonio Frederick,85,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,...,480,2020-05-03,Penicillin,Abnormal,1938-09-16,False,False,orthopedics,inpatient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
4,elective,Antonio Frederick,85,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,...,480,2020-05-03,Penicillin,Abnormal,1938-09-16,False,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,elective,Michelle Robinson MD,67,Female,B-,Cancer,2019-02-05,Paul Walker,Mills-Montgomery,Cigna,...,172,2019-02-09,Lipitor,Abnormal,1956-05-13,True,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
995,elective,Michelle Robinson MD,67,Female,B-,Cancer,2019-02-05,Paul Walker,Mills-Montgomery,Cigna,...,172,2019-02-09,Lipitor,Abnormal,1956-05-13,True,False,ophthalmology,day_patient,"[Eye exam, Tonometry (eye pressure test), Reti..."
996,elective,Michelle Robinson MD,67,Female,B-,Cancer,2019-02-05,Paul Walker,Mills-Montgomery,Cigna,...,172,2019-02-09,Lipitor,Abnormal,1956-05-13,True,False,ophthalmology,outpatient,"[Eye exam, Tonometry (eye pressure test), Reti..."
997,elective,Michelle Robinson MD,67,Female,B-,Cancer,2019-02-05,Paul Walker,Mills-Montgomery,Cigna,...,172,2019-02-09,Lipitor,Abnormal,1956-05-13,True,False,geriatrics,outpatient,"[Comprehensive geriatric assessment, Blood tes..."


In [10]:
"""
STEP 3 
    Create a rowq ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
    
"""

df =  df.withColumn('stay_name', 
                   when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
                   .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
                   .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
                   .otherwise(col('name'))
                  ).withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))


# Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [11]:
ranked_df.createGlobalTempView("ranked_df")

unique_dobs_df = spark.sql("""
WITH NameCounts AS (
    SELECT name
    FROM global_temp.ranked_df
)

SELECT DISTINCT r.name, r.unique_id
FROM global_temp.ranked_df r
JOIN NameCounts n ON r.name = n.name
ORDER BY r.name
LIMIT 10
""")

24/02/12 20:39:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
from pyspark.sql import DataFrame

# COSTLY WAY TO VERIFY THAT THE RANK WORKS BELOW.... 

def verify_ranking_counts(_df: DataFrame, ranked_df: DataFrame, names: list, unique_ids: list):
    for name, unique_id in zip(names, unique_ids):
        # Define filter conditions
        name_condition = col("name") == name
        unique_id_condition = col("unique_id") == unique_id
        stay_name_condition = col("stay_name").like(f"{name}_%")

        df_filtered_count = (_df
                             .filter(name_condition & unique_id_condition & stay_name_condition)
                             .count())

        ranked_df_filtered_count = (ranked_df
                                    .filter(name_condition & unique_id_condition & stay_name_condition)
                                    .count())

        # Assert that the counts are equal for each name-unique_id pair
        assert df_filtered_count == ranked_df_filtered_count, f"The number of matching rows for {name} with unique ID {unique_id} should be the same in both DataFrames."

# Example usage:
unique_names = [row['name'] for row in unique_dobs_df.select("name").collect()]
unique_ids = [row['unique_id'] for row in unique_dobs_df.select("unique_id").collect()]
verify_ranking_counts(df, ranked_df, unique_names, unique_ids)

In [13]:
def verify_ranking(_df: DataFrame, ranked_df: DataFrame):
    joined_df = _df.alias("df").join(
        ranked_df.alias("ranked"),
       
       (col("df.unique_id") == col("ranked.unique_id")),
        "inner"
    )

    verification_df = joined_df.select(
       col("df.unique_id")
    ).distinct().groupBy("unique_id").agg(
        count("unique_id").alias("unique_id_count")
    )

    # Check if any name has a mismatch in the count of unique IDs
    mismatch_count = verification_df.filter("unique_id_count > 1").count()

    # Assert that there are no mismatches
    assert mismatch_count == 0, f"There are names with mismatched unique ID counts. mismatch_count: {mismatch_count}"

verify_ranking(df, ranked_df)


In [14]:
not_geriatric_df = ranked_df.withColumn("is_geriatric", is_geriatric).where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))

In [15]:
not_geriatric_df.show()

+-------------------+--------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+-----------+------------+----------+---------+------------+-------------------+----------+--------------------+--------------------+--------------------+-------+------------+
|top_level_admission|          name|Age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|discharge_date| medication|test_results|       DOB|is_female|is_pediatric|sub_level_admission| stay_type|      possible_tests|           stay_name|           unique_id|row_num|is_geriatric|
+-------------------+--------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+-----------+------------+----------+---------+-------

In [16]:
"""
Step 4: 
    Sort out geriatrics data and verify that individuals are not geriatrics 
"""
ranked_df = ranked_df.withColumn("is_geriatric", is_geriatric)

not_geriatric_df = ranked_df.where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))

ranked_tot = get_row_count(ranked_df)
geriatric_tot = get_row_count(not_geriatric_df)

filtered_df = ranked_df.join(not_geriatric_df, on= ['sub_level_admission', 'DOB'], how="left_anti")

filtered_tot = get_row_count(filtered_df)

assert filtered_tot == ranked_tot - geriatric_tot, "Row counts do not match expected value"

# checks the above calculation
assert get_row_count(filtered_df.where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))) == 0

In [17]:
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 

In [18]:
assert get_row_count(ranked_df.where((col("gender") == "Male") & (col("is_female") == True))) == 0

not_female_df = filtered_df.where((col("sub_level_admission").isin(female_only)) & (col("is_female") == False))

assert  not_female_df.where((col("gender") == "Female") & (col("is_female") == True)).count() == 0

In [19]:
not_female_tot = get_row_count(not_female_df)

filtered_df_female = filtered_df.join(not_female_df, on=['sub_level_admission', 'DOB'], how = 'left_anti')

filtered_female_tot = get_row_count(filtered_df_female)

assert filtered_df_female.where((col("gender") == "Male") & (col("sub_level_admission").isin(female_only))).count() == 0 


In [20]:
# filter pediatric patients who cannot be pregnant (based on legal age in the UK, 16) No assumption made an individual cannot choose to get pregnant before this age. 
under16_female_df = filtered_df_female.where((col("Age") <16) & (col("sub_level_admission") == 'maternity')).orderBy("Age")

In [21]:
filtered_df_female_u16 = filtered_df_female.join(under16_female_df, on=['sub_level_admission', 'DOB'], how = 'left_anti')

filtered_female_tot_u16 = get_row_count(under16_female_df)

assert filtered_df_female_u16.where((col("sub_level_admission") == "maternity") & (col("Age") <16)).count() == 0 

In [25]:
df = filtered_df_female_u16
split_col = split(df['name'], ' ')

df_renamed = (df.withColumn('re_name', 
                    when(size(split_col) == 4, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2).substr(1, 1), lit('. '), split_col.getItem(3)))
                    .when(size(split_col) == 3, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2)))
                    .otherwise(
                        concat(split_col.getItem(0).substr(1, 1), lit('. '), split_col.getItem(1))
                               )
                    )
      )


In [26]:
df_renamed.select(['name', 're_name']).show(n=2000)

+--------------------+--------------+
|                name|       re_name|
+--------------------+--------------+
|         Scott Adams|      S. Adams|
|         Scott Adams|      S. Adams|
|        Joy Mitchell|   J. Mitchell|
|        Joy Mitchell|   J. Mitchell|
|      Malik Phillips|   M. Phillips|
|      Malik Phillips|   M. Phillips|
|      Jonathan Moore|      J. Moore|
|      Jonathan Moore|      J. Moore|
|     Christina Perry|      C. Perry|
|     Christina Perry|      C. Perry|
|    Brandon Thornton|   B. Thornton|
|    Brandon Thornton|   B. Thornton|
|         David Allen|      D. Allen|
|         David Allen|      D. Allen|
|          Jamie Holt|       J. Holt|
|          Jamie Holt|       J. Holt|
|        Penny Parker|     P. Parker|
|       Adriana Ochoa|      A. Ochoa|
|       Adriana Ochoa|      A. Ochoa|
|        Miranda Frey|       M. Frey|
|        Miranda Frey|       M. Frey|
|        Brian Torres|     B. Torres|
|        Brian Torres|     B. Torres|
|        Bri

In [ ]:
# TODO: drop full name for initial and surname since nname column is poor. 
# adjust doctor column 
# medical condition 
# admission date to be checked again dob, 
# medication to be checked 
# choose random results 

# drop stay_name and unique_id 


In [ ]:
# Filter to keep only the top-ranked row within each partition
ranked_df = filtered_df_female

In [ ]:
ranked_df.where(col("name") == "Daniel Mccoy").show(n=8000)

In [ ]:
from pyspark.sql.types import StringType

df = filtered_df_female.select([col(c).cast(StringType()).alias(c) for c in df.columns])
df.write.csv('./temp_data/female/filtered_df_female.csv', mode = 'overwrite', header=True)

In [ ]:
# TODO initialise first name to make data more realistic for name columns i.e. Daniel Mccoy is seen a Male name but here its Female

In [ ]:
spark.stop()